In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
import re
import importlib
from functions import *
import random
import folium
from GA import *
from shapely.geometry import Point , Polygon

# 1) DATASETS and Data Cleaning

## a) Population Dataset

In [3]:
population_path = "/home/saydam/Desktop/2024-2025_itu/yzv202/project/github/datasets/population_with_coordinates.csv"

In [4]:
raw_population_df = pd.read_csv(population_path , header=None, sep=",")

In [5]:
raw_population_df = raw_population_df.dropna(how='all', axis=1)

In [6]:
raw_population_df.columns = raw_population_df.iloc[0]

In [7]:
raw_population_df = raw_population_df.drop(index=0).reset_index(drop=True)
raw_population_df = raw_population_df.loc[:, raw_population_df.columns.notna()]

In [8]:
raw_population_df[['latitude', 'longitude']] = raw_population_df['coordinate'].str.split(',', expand=True)

In [9]:
raw_population_df['latitude'] = raw_population_df['latitude'].astype(float)
raw_population_df['longitude'] = raw_population_df['longitude'].astype(float)
raw_population_df['population'] = raw_population_df['population'].astype(int)

In [10]:
population_df = raw_population_df[['population', 'neighborhood', 'township', 'neighborhood_code', 'latitude', 'longitude']]

In [11]:
population_df = population_df.dropna()

In [12]:
population_df.head()

,population,neighborhood,township,neighborhood_code,latitude,longitude
0,4693,Adnan Menderes,Arnavutköy,40490,41.211179,28.700163
1,47828,Anadolu,Arnavutköy,99359,41.186036,28.749463
2,23116,Arnavutköy Merkez,Arnavutköy,40478,41.182546,28.737891
3,10566,Atatürk,Arnavutköy,40482,41.190149,28.760125
4,12829,Boğazköy İstiklal,Arnavutköy,40483,41.183488,28.768235


In [13]:
population_df.describe()

,population,latitude,longitude
count,708.000000,708.000000,708.000000
mean,21397.411017,41.029966,28.972329
std,15943.770157,0.071404,0.193073
min,2025.000000,40.817868,28.409171
25%,10279.250000,40.998081,28.854917
50%,17970.000000,41.028968,28.981120
75%,28527.750000,41.063456,29.109349
max,112367.000000,41.269994,29.386538


## b) Metro Station Dataset

In [14]:
stations_gdf = gpd.read_file("/home/saydam/Desktop/2024-2025_itu/yzv202/project/github/datasets/station.geojson")

In [15]:
print(stations_gdf.head())

                   ISTASYON                                      PROJE_ADI  \
0               Mehmet Akif            T1 Kabataş - Bağcılar Tramvay Hattı   
1                  Soğanlık                   M4 Kadıköy - SGH Metro Hattı   
2                   Aksaray  M1A Yenikapı - Atatürk Havalimanı Metro Hattı   
3                 Olimpiyat            M9 Bahariye - Olimpiyat Metro Hattı   
4  Sağmalcılar-Pancar Motor      T4 Topkapı - Mescid-i Selam Tramvay Hattı   

               PROJE_ASAMA HAT_TURU                                  MUDURLUK  \
0  Mevcut Hattaki İstasyon  Tramvay   Avrupa Yakası Raylı Sistemler Müdürlüğü   
1  Mevcut Hattaki İstasyon    Metro  Anadolu Yakası Raylı Sistemler Müdürlüğü   
2  Mevcut Hattaki İstasyon    Metro   Avrupa Yakası Raylı Sistemler Müdürlüğü   
3  Mevcut Hattaki İstasyon    Metro   Avrupa Yakası Raylı Sistemler Müdürlüğü   
4  Mevcut Hattaki İstasyon  Tramvay   Avrupa Yakası Raylı Sistemler Müdürlüğü   

                    geometry  
0   POINT (28

In [16]:
stations_gdf["lat"] = stations_gdf.geometry.x
stations_gdf["lon"] = stations_gdf.geometry.y

In [17]:
stations_df = stations_gdf[["ISTASYON", "PROJE_ADI", "HAT_TURU", "lat", "lon"]]

In [18]:
stations_df[["lat", "lon"]] = stations_df[["lon", "lat"]]

/tmp/ipykernel_86639/3141347488.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stations_df[["lat", "lon"]] = stations_df[["lon", "lat"]]


In [19]:
stations_df.head()

,ISTASYON,PROJE_ADI,HAT_TURU,lat,lon
0,Mehmet Akif,T1 Kabataş - Bağcılar Tramvay Hattı,Tramvay,41.005800,28.881681
1,Soğanlık,M4 Kadıköy - SGH Metro Hattı,Metro,40.913288,29.192398
2,Aksaray,M1A Yenikapı - Atatürk Havalimanı Metro Hattı,Metro,41.012008,28.948090
3,Olimpiyat,M9 Bahariye - Olimpiyat Metro Hattı,Metro,41.079466,28.767234
4,Sağmalcılar-Pancar Motor,T4 Topkapı - Mescid-i Selam Tramvay Hattı,Tramvay,41.057280,28.906940


# Visulation
Initial maps

In [20]:
istanbul_coords = [41.0082, 28.9784]


map_istanbul = folium.Map(location=istanbul_coords, zoom_start=11, tiles='CartoDB positron')

for index, row in stations_df.iterrows():
    popup_text = f"{row['ISTASYON']}<br>{row['PROJE_ADI']}"
    folium.Marker(
        location=[row['lat'], row['lon']],
        popup=popup_text,
        icon=folium.Icon(color='blue' if row['HAT_TURU'] == 'Metro' else 'green', icon='train')
    ).add_to(map_istanbul)

map_istanbul.save("initial_metro_lines.html")

In [21]:
m = folium.Map(location=istanbul_coords, zoom_start=10 , tiles='CartoDB positron')


for _, row in population_df.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f"{row['neighborhood']} ({row['township']})\nNüfus: {row['population']}",
        tooltip=row['neighborhood']
    ).add_to(m)

m.save("neighborhood.html")

# Calculating grids
These are going to be candidate stations

In [30]:
istanbul_poly = Polygon([
    (28.6, 41.2), 
    (29.0, 41.3),
    (29.4, 41.3),
    (29.8, 41.0),
    (29.3, 40.9),
    (28.8, 40.8),
    (28.6, 41.0)
])

grid_list = create_grid_for_polygon(istanbul_poly)

In [31]:
candidate_stations_df = pd.DataFrame(grid_list)
candidate_stations_df.head()

,station_id,lat,lon
0,1,28.695797,40.987937
1,2,29.408903,41.172191
2,3,29.383560,41.139977
3,4,29.376912,41.234437
4,5,29.321430,41.142090


In [34]:
candidate_stations_df.describe()

,station_id,lat,lon
count,1000.000000,1000.000000,1000.000000
mean,500.500000,29.105650,41.069787
std,288.819436,0.279474,0.116950
min,1.000000,28.604162,40.807063
25%,250.750000,28.869751,40.980837
50%,500.500000,29.093302,41.064432
75%,750.250000,29.328894,41.164802
max,1000.000000,29.767187,41.298836


In [35]:
candidate_stations_df['TYPE'] = 'candidate'
stations_df['TYPE'] = 'existing'

/tmp/ipykernel_86639/535197162.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stations_df['TYPE'] = 'existing'


In [36]:
stations_df.head()

,ISTASYON,PROJE_ADI,HAT_TURU,lat,lon,TYPE
0,Mehmet Akif,T1 Kabataş - Bağcılar Tramvay Hattı,Tramvay,41.005800,28.881681,existing
1,Soğanlık,M4 Kadıköy - SGH Metro Hattı,Metro,40.913288,29.192398,existing
2,Aksaray,M1A Yenikapı - Atatürk Havalimanı Metro Hattı,Metro,41.012008,28.948090,existing
3,Olimpiyat,M9 Bahariye - Olimpiyat Metro Hattı,Metro,41.079466,28.767234,existing
4,Sağmalcılar-Pancar Motor,T4 Topkapı - Mescid-i Selam Tramvay Hattı,Tramvay,41.057280,28.906940,existing


In [37]:
candidate_stations_df.head()

,station_id,lat,lon,TYPE
0,1,28.695797,40.987937,candidate
1,2,29.408903,41.172191,candidate
2,3,29.383560,41.139977,candidate
3,4,29.376912,41.234437,candidate
4,5,29.321430,41.142090,candidate


In [38]:
all_stations = pd.concat([
    candidate_stations_df[['station_id', 'lat', 'lon' , 'TYPE']],
    stations_df[['ISTASYON', 'lat', 'lon' , 'TYPE']].rename(columns={'ISTASYON': 'station_id'})
]).reset_index(drop=True)


In [39]:
all_stations = all_stations.reset_index(drop=True)
all_stations['station_id'] = all_stations.index + 34001

In [40]:
all_stations.head()

,station_id,lat,lon,TYPE
0,34001,28.695797,40.987937,candidate
1,34002,29.408903,41.172191,candidate
2,34003,29.383560,41.139977,candidate
3,34004,29.376912,41.234437,candidate
4,34005,29.321430,41.142090,candidate


In [41]:
all_stations_pop = calculate_population_per_station(all_stations , population_df)

In [35]:
all_stations_pop_filtered = all_stations_pop[
    ~((all_stations_pop['arrived_population'] < 1000) & (all_stations_pop['TYPE'] == 'candidate'))
]

all_stations_pop_filtered = all_stations_pop_filtered.sort_values(by='arrived_population' , ascending = False)

In [36]:
all_stations_pop_filtered.head()

,station_id,lat,lon,TYPE,arrived_population
164,34165,41.006883,28.845000,candidate,302574
134,34135,41.001450,28.845000,candidate,294879
104,34105,40.996017,28.845000,candidate,261326
225,34226,41.017750,28.855000,candidate,259554
1177,35178,41.019391,28.857845,existing,235721


In [37]:
all_stations_pop_filtered.describe()

,station_id,lat,lon,arrived_population
count,866.000000,866.000000,866.000000,866.000000
mean,34654.267898,41.034148,28.910335,71616.110855
std,383.173053,0.055695,0.126874,48453.873690
min,34001.000000,40.784064,28.670765,0.000000
25%,34318.250000,41.006883,28.825000,34711.500000
50%,34605.500000,41.034050,28.895000,66030.500000
75%,35026.750000,41.068220,28.965000,102849.500000
max,35243.000000,41.256408,29.409966,302574.000000


### Extracting Metro Lines and Stations
We have not a extract metro lines and their stations dictionary. For further operation we need this dict .

In [38]:
TOLERANCE = 0.0005

project_dict = defaultdict(list)

existing_stations = all_stations_pop[all_stations_pop['TYPE'] == 'existing']

for idx, row in stations_df.iterrows():
    proje_adi_full = row['PROJE_ADI']
    lat = row['lat']
    lon = row['lon']

    match_hat = re.search(r'\b(M\d+[A-Z]?|T\d+)\b', proje_adi_full)
    if not match_hat:
        continue  
    hat_kodu = match_hat.group()


    match = existing_stations[
        (existing_stations['lat'].sub(lat).abs() < TOLERANCE) &
        (existing_stations['lon'].sub(lon).abs() < TOLERANCE)
    ]

    if not match.empty:
        for station_id in match['station_id']:
            project_dict[hat_kodu].append(station_id)

project_dict = dict(project_dict)


In [39]:
print(project_dict)

{'T1': [34901, 34917, 34925, 34927, 34931, 34932, 34933, 34955, 34956, 34965, 34966, 34971, 34975, 34994, 34995, 34997, 35001, 35007, 35025, 35026, 35027, 35028, 35031, 35035, 35050, 35051, 35054, 35085, 35093, 35096, 35109, 35124], 'M4': [34902, 34909, 34910, 34938, 34991, 34999, 35010, 35012, 35029, 35034, 35039, 35053, 35057, 35024, 35058, 35062, 35065, 35081, 35086, 35092, 35161, 35162, 35163, 35164, 35165, 35166, 35167, 35168, 35169, 35170], 'M1A': [34903, 34941, 34945, 34947, 34958, 34961, 34969, 34998, 35003, 35033, 35048, 35083, 35084, 35090, 35097, 35099, 35111, 35112, 35114], 'M9': [34904, 34984, 35087, 35060, 35130, 35150, 35183, 35184, 35185, 35186, 35187, 35188, 35226, 35189, 35014, 35190, 35191], 'T4': [34905, 34906, 34923, 34940, 34942, 34948, 34964, 34967, 34968, 34972, 34973, 34974, 34996, 35002, 35008, 35032, 35056, 35063, 35091, 35098, 35110, 35115], 'M5': [34912, 34914, 34979, 35011, 35015, 35018, 35041, 35044, 35047, 35067, 35071, 35073, 35075, 35076, 35077, 35100,

we dont know last stations of metro lines. i added last station end of the metro line array.

In [40]:
existing_stations_dict = {'T1': [34901, 34917, 34925, 34927, 34931, 34932, 34933, 34955, 34956, 34965, 34966, 34971, 34975, 34994, 34995, 
34997, 35001, 35025, 35026, 35027, 35028, 35031, 35035, 35050, 35051, 35054, 35085, 35093, 35096, 35109, 35124 , 35007], 
                            'M4': [34902, 34909, 34910, 34938, 34991, 34999, 35010, 35012, 35029, 35034, 35039, 35053, 35057, 35024, 
35058, 35062, 35065, 35081, 35086, 35092, 35161, 35162, 35163, 35164, 35165, 35166, 35168, 35169, 35170 , 35167], 
                          'M1A': [34903, 34941, 34945, 34947, 34958, 34961, 34969, 
34998, 35003, 35033, 35083, 35084, 35090, 35097, 35099, 35111, 35112, 35114 , 35048], 
                          'M9': [34984, 35087, 35060, 35130, 35150, 35183,
35184, 35185, 35186, 35187, 35188, 35226, 35189, 35014, 35190, 35191, 34904], 
                          'T4': [34905, 34906, 34923, 34940, 34942, 34948, 34964, 34967, 34968,
34972, 34973, 34974, 34996, 35002, 35008, 35032, 35056, 35063, 35091, 35098, 35110, 35115], 
                          'M5': [34912, 34914, 34979, 35011, 35015, 35018,
35041, 35044, 35047, 35067, 35071, 35073, 35075, 35076, 35077, 35100, 35117, 35154, 35155, 35157, 35158, 35159, 35160, 35156], 
                          'M7': [34915,
34937, 34952, 34953, 34978, 34980, 34982, 35013, 35016, 35021, 35043, 35046, 35070, 35103, 35107, 35119, 35120, 35121, 35201, 35202,
35203, 35204, 35205, 35206, 35207, 35208, 35209, 35210, 35242 , 35122],
                          'T5': [34918, 34919, 34920, 34922, 34939, 34959, 35055, 35125, 35126, 35127, 
35182, 35214, 35243], 
                          'M2': [34924, 35116, 34930, 34944, 34960, 34963, 34989, 34990, 35005, 35009, 35049, 35064, 35066, 35082, 35094, 
                                 34924, 35116, 35123 , 35004], 
                           'M3': [34926, 34957, 34928, 34943, 34970, 35006, 35037, 35060, 35130,  35089, 35174, 35177, 35178, 35180, 
35181, 35211, 35212, 35227, 35228, 35229, 35230 , 35061], 
                          'M6': [34949, 35059, 35088 , 35040], 
                          'M1B': [34926, 34957, 35036, 35038, 35052, 35095, 35175, 35176, 35179, 35213, 35216,  35221, 35222, 35188, 
35226 , 35219], 
                          'M8': [35118, 35171, 35173, 35192, 35193, 35194, 35195, 35196, 35197, 35198, 35199, 35200 , 35172], 
                          'M11': [35140, 35141, 35142, 
35143, 35144, 35145, 35146, 35147, 35218, 35220, 35223, 35224, 35225 , 35215], 
                          'M12': [35231, 35232, 35233, 35234, 
35235, 35236, 35237, 35238, 35239, 35240, 35241]}


ww should calculate the connectivitiy dictionary . this dict shows which station could be connected whichs stations 

In [41]:
connectivity_dict = calculate_connectivity_dict(all_stations_pop)

In [42]:
print(len(connectivity_dict))

1240


# GA

In [72]:
GeneticAlgorithm = GeneticMetroPlanner(
    all_stations_df = all_stations_pop_filtered,
    connectivity_dict = connectivity_dict,
    existing_lines_dict = existing_stations_dict)

In [73]:
chromosome = GeneticAlgorithm.generate_chromosome()
for line, stations in chromosome.items():
    print(f"{line}: {len(stations)} station")

T1: 34 station
M4: 30 station
M1A: 22 station
M9: 17 station
T4: 22 station
M5: 24 station
M7: 31 station
T5: 15 station
M2: 19 station
M3: 24 station
M6: 7 station
M1B: 19 station
M8: 13 station
M11: 16 station
M12: 11 station


In [74]:
for line , stations in existing_stations_dict.items():
    print(f"{line}:  {len(stations)} station")

T1:  32 station
M4:  30 station
M1A:  19 station
M9:  17 station
T4:  22 station
M5:  24 station
M7:  30 station
T5:  13 station
M2:  19 station
M3:  22 station
M6:  4 station
M1B:  16 station
M8:  13 station
M11:  14 station
M12:  11 station


In [47]:
print(chromosome)

{'T1': [34901, 34917, 34925, 34927, 34931, 34932, 34933, 34955, 34956, 34965, 34966, 34971, 34975, 34994, 34995, 34997, 35001, 35025, 35026, 35027, 35028, 35031, 35035, 35050, 35051, 35054, 35085, 35093, 35096, 35109, 35124, 35007, 34405, 34497], 'M4': [34902, 34909, 34910, 34938, 34991, 34999, 35010, 35012, 35029, 35034, 35039, 35053, 35057, 35024, 35058, 35062, 35065, 35081, 35086, 35092, 35161, 35162, 35163, 35164, 35165, 35166, 35168, 35169, 35170, 35167], 'M1A': [34903, 34941, 34945, 34947, 34958, 34961, 34969, 34998, 35003, 35033, 35083, 35084, 35090, 35097, 35099, 35111, 35112, 35114, 35048], 'M9': [34984, 35087, 35060, 35130, 35150, 35183, 35184, 35185, 35186, 35187, 35188, 35226, 35189, 35014, 35190, 35191, 34904], 'T4': [34905, 34906, 34923, 34940, 34942, 34948, 34964, 34967, 34968, 34972, 34973, 34974, 34996, 35002, 35008, 35032, 35056, 35063, 35091, 35098, 35110, 35115], 'M5': [34912, 34914, 34979, 35011, 35015, 35018, 35041, 35044, 35047, 35067, 35071, 35073, 35075, 35076,

In [67]:
m_n = visualize_chromosome(chromosome , all_stations)

In [68]:
m_n.save("chromosome_map_new.html")

In [51]:
m_old = visualize_chromosome(existing_stations_dict , all_stations)
m_old.save("chromosome_map_old.html")

In [52]:
print(GeneticAlgorithm.calculate_population_for_chromosome(chromosome))

18821358


In [53]:
GeneticAlgorithm.generate_initial_population()

In [54]:
print(len(GeneticAlgorithm.population))

10


In [55]:
GeneticAlgorithm.fitness_population()

In [56]:
print(GeneticAlgorithm.fitness_values)

[np.int64(19432534), np.int64(18833850), np.int64(19028021), np.int64(20017027), np.int64(19241998), np.int64(19601074), np.int64(19403789), np.int64(19309268), np.int64(19183988), np.int64(18701182)]


In [57]:
planner = GeneticMetroPlanner(
    all_stations_df=all_stations,
    connectivity_dict=connectivity_dict,
    existing_lines_dict=existing_stations_dict,
    mutation_rate=0.2,
    generation_number=10,
    child_number=20,
    new_station_number=50,
    max_per_station=5
)


In [58]:
best_solution, best_score = planner.run()
print("Best solution")
print("Arrived population", best_score)

for line, stations in best_solution.items():
    print(f"{line}: {stations}")

Best solution
Arrived population 23819672
T1: [34901, 34917, 34925, 34927, 34931, 34932, 34933, 34955, 34956, 34965, 34966, 34971, 34975, 34994, 34995, 34997, 35001, 35025, 35026, 35027, 35028, 35031, 35035, 35050, 35051, 35054, 35085, 35093, 35096, 35109, 35124, 35007, 34228, 34226, 34347, 34379, 34290, 34172, 34234, 34356, 34359, 34296, 34418, 34478, 34388, 34449]
M4: [34902, 34909, 34910, 34938, 34991, 34999, 35010, 35012, 35029, 35034, 35039, 35053, 35057, 35024, 35058, 35062, 35065, 35081, 35086, 35092, 35161, 35162, 35163, 35164, 35165, 35166, 35168, 35169, 35170, 35167]
M1A: [34903, 34941, 34945, 34947, 34958, 34961, 34969, 34998, 35003, 35033, 35083, 35084, 35090, 35097, 35099, 35111, 35112, 35114, 35048, 34044, 34101, 34133, 34012, 34009, 34036, 34094]
M9: [34984, 35087, 35060, 35130, 35150, 35183, 35184, 35185, 35186, 35187, 35188, 35226, 35189, 35014, 35190, 35191, 34904, 34426, 34395, 34424, 34453, 34543, 34425, 34393, 34365, 34307, 34250, 34309]
T4: [34905, 34906, 34923, 3

In [59]:
m_best = visualize_chromosome(best_solution , all_stations)

In [60]:
m_best.save("best_metro_lines.html")

In [64]:
istanbul_poly = Polygon([
    (28.6, 41.2),  # kabaca İstanbul'un batısı
    (29.0, 41.3),
    (29.4, 41.3),
    (29.8, 41.0),
    (29.3, 40.9),
    (28.8, 40.8),
    (28.6, 41.0)
])

points_inside = create_grid_for_polygon(istanbul_poly)

In [65]:
print(len(points_inside))

1000


In [71]:
visualize_chromosome(points_inside , all_stations)

AttributeError: 'list' object has no attribute 'items'